In [3]:
!pip install -q open3d
 # Pillow 꼭 버전 지킬 것. 함수가 달라져서 민감함.
!pip install Pillow==9.0.0 -q

In [4]:
%pwd

'C:\\Dev\\Code sources\\VisualStudioCode repos\\salmonsush\\HoHoNet-dollhouse'

In [6]:
!conda env list

# conda environments:
#
base                     C:\Dev\Miniconda
FreDS                    C:\Dev\Miniconda\envs\FreDS
depth_1.7                C:\Dev\Miniconda\envs\depth_1.7
firstvenv                C:\Dev\Miniconda\envs\firstvenv
hoho                  *  C:\Dev\Miniconda\envs\hoho
o3d                      C:\Dev\Miniconda\envs\o3d
panoformer               C:\Dev\Miniconda\envs\panoformer



In [7]:
!pip install yacs -q
!pip install gdown -q
!pip install plotly -q

In [9]:
import os, sys, time
import argparse
import importlib
from tqdm.notebook import tqdm
# from imageio import imread
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2


ModuleNotFoundError: No module named 'torch'

### Download pretrained
- We use HoHoNet w/ hardnet encoder in this demo
- Download other version [here](https://drive.google.com/drive/folders/1raT3vRXnQXRAQuYq36dE-93xFc_hgkTQ?usp=sharing)

In [10]:
PRETRAINED_PTH = 'ckpt/mp3d_layout_HOHO_layout_aug_efficienthc_Transen1_resnet34/ep300.pth'

if not os.path.exists(PRETRAINED_PTH):
    os.makedirs(os.path.split(PRETRAINED_PTH)[0], exist_ok=True)
    !gdown 'https://drive.google.com/uc?id=1OU9uyuNiswkPovJuvG3sevm3LqHJgazJ' -O $PRETRAINED_PTH

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id='https://drive.google.com/uc?id=1OU9uyuNiswkPovJuvG3sevm3LqHJgazJ' 



### Load model config
- We use HoHoNet w/ hardnet encoder in this demo
- Find out other version in `mp3d_depth/` and `s2d3d_depth`

In [6]:
from lib.config import config

config.defrost()
config.merge_from_file('config/mp3d_layout/HOHO_layout_aug_efficienthc_Transen1_resnet34.yaml')
config.freeze()

### Load model

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('devcie:', device)

model_file = importlib.import_module(config.model.file)
model_class = getattr(model_file, config.model.modelclass)
net = model_class(**config.model.kwargs)
net.load_state_dict(torch.load(PRETRAINED_PTH, map_location=device))
net = net.eval().to(device)

devcie: cpu


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Load image

In [8]:
rgb1 = imread('/content/drive/MyDrive/Capstone/input/006/crop_pano/44304216.png')

# plt.imshow(rgb1)
# plt.show()
rgb1 = cv2.resize(rgb1, (1024, 512), interpolation=cv2.INTER_LINEAR)


<ipython-input-8-42cef4c937a3>:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  rgb1 = imread('/content/drive/MyDrive/Capstone/input/006/crop_pano/44304216.png')


In [9]:
rgb2 = imread('/content/drive/MyDrive/Capstone/input/006/crop_pano/4898.png')

# plt.imshow(rgb2)/
# plt.show()
rgb2 = cv2.resize(rgb2, (1024, 512), interpolation=cv2.INTER_LINEAR) # 문좌표 135,480


<ipython-input-9-960d22a6a66e>:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  rgb2 = imread('/content/drive/MyDrive/Capstone/input/006/crop_pano/4898.png')


In [19]:
rgb3 = imread('/content/drive/MyDrive/Capstone/input/006/crop_pano/4955.png')                                                                                   # 3버어어어어ㅓ언어ㅓ엉ㄴ

# plt.imshow(rgb2)/
# plt.show()
rgb3 = cv2.resize(rgb3, (1024, 512), interpolation=cv2.INTER_LINEAR) # 문좌표 135,480


<ipython-input-19-1b08d248d810>:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  rgb3 = imread('/content/drive/MyDrive/Capstone/input/006/crop_pano/4955.png')                                                                                   # 3버어어어어ㅓ언어ㅓ엉ㄴ


In [11]:
import open3d as o3d
import plotly.graph_objects as go
from scipy.signal import correlate2d
from scipy.ndimage import shift
from skimage.transform import resize
from eval_layout import layout_2_depth
from lib.misc.post_proc import np_coor2xy, np_coorx2u, np_coory2v

def Infer(rgb, door_x, door_y, net):
    """
    infer and exports point and faces and resized point of door.
    output:{
      points : all the spherical-coordinate(x,y,z) of 131528 points(shape: 131528,3) on the image, 
      faces : all the 3-indices(i,j,k) of vertex for each  260000 faces, 
      cor_id : 3-coordinate(x,y,z) of cor_id, the vertices of layout i.e 12, (2x6) points, 
      door_x_resized : resized x-coordinate of door point, 
      door_y_resized : resized y-coordinate of door point, 
      xs : spherical-coordinate(x) of 131528 points(shape: 256,513), 
      ys : spherical-coordinate(y) of 131528 points(shape: 256,513),
      zs : spherical-coordinate(z) of 131528 points(shape: 256,513)
    }
    """
    # Move image into tensor, normalize to [0,255], resize to 512x1024
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    x = torch.from_numpy(rgb).permute(2,0,1)[None].float() / 255.
    if x.shape[2:] != (512, 1024):
        x = torch.nn.functional.interpolate(x, size=(512, 1024), mode='area')
        print("fitting")
    x = x.to(device)
    
    # Model feedforward
    with torch.no_grad():
        ts = time.time()
        layout = net.infer(x)
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        print(f'Eps time: {time.time() - ts:.2f} sec.')

    cor_id = layout['cor_id']
    y_bon_ = layout['y_bon_']
    y_cor_ = layout['y_cor_']

    # source code
    H, W = 256, 512

    # resize door coordinate from 512x1024 to 256x512
    door_x_resized = int(door_x*(W/1024))
    door_y_resized = int(door_y*(H/512))
    print(door_x, door_y)


    ignore_floor = False
    ignore_ceiling = False
    ignore_wall = False

    # Convert corners to layout
    depth, floor_mask, ceil_mask, wall_mask = [
        resize(v, [H, W], order=0, preserve_range=True).astype(v.dtype)
        for v in layout_2_depth(cor_id, *rgb.shape[:2], return_mask=True)]

    cor_id[:,0] = (cor_id[:,0] * (256/512)).astype(int)
    cor_id[:,1] = (cor_id[:,1] * (512/1024)).astype(int)
    cor_id = cor_id[:,::-1].astype(int)

    # 빈 좌표계 만들어주기
    coorx, coory = np.meshgrid(np.arange(W), np.arange(H)) #256,512
    # 구면좌표계로 변환
    us = -1 * np_coorx2u(coorx, W) # -pi ~ pi
    vs = np_coory2v(coory, H) # -1/2*pi ~ 1/2*pi
    zs = depth * np.sin(vs) 
    cs = depth * np.cos(vs)
    xs = cs * np.sin(us)
    ys = -cs * np.cos(us) # 전부다 사이즈 256.512

    # Aggregate mask
    mask = np.ones_like(floor_mask)
    if ignore_floor:
        mask &= ~floor_mask
    if ignore_ceiling:
        mask &= ~ceil_mask
    if ignore_wall:
        mask &= ~wall_mask

    # Prepare ply's points and faces
    xyzrgb = np.concatenate([
        xs[...,None], ys[...,None], zs[...,None],
        resize(rgb, [H, W])], -1)
    xyzrgb = np.concatenate([xyzrgb, xyzrgb[:,[0]]], 1) # 수평선으로 폐쇄된 mesh를 만들기 위한 것.

    mask = np.concatenate([mask, mask[:,[0]]], 1)

    lo_tri_template = np.array([
        [0, 0, 0],
        [0, 1, 0],
        [0, 1, 1]])
    up_tri_template = np.array([
        [0, 0, 0],
        [0, 1, 1],
        [0, 0, 1]])
    ma_tri_template = np.array([
        [0, 0, 0],
        [0, 1, 1],
        [0, 1, 0]])
    lo_mask = (correlate2d(mask, lo_tri_template, mode='same') == 3)
    up_mask = (correlate2d(mask, up_tri_template, mode='same') == 3)
    ma_mask = (correlate2d(mask, ma_tri_template, mode='same') == 3) & (~lo_mask) & (~up_mask)
    ref_mask = (
        lo_mask | (correlate2d(lo_mask, np.flip(lo_tri_template, (0,1)), mode='same') > 0) |\
        up_mask | (correlate2d(up_mask, np.flip(up_tri_template, (0,1)), mode='same') > 0) |\
        ma_mask | (correlate2d(ma_mask, np.flip(ma_tri_template, (0,1)), mode='same') > 0)
    ) # ref_mask : 256x513
    points = xyzrgb[ref_mask]
    ref_id = np.full(ref_mask.shape, -1, np.int32)
    ref_id[ref_mask] = np.arange(ref_mask.sum())

    faces_lo_tri = np.stack([
        ref_id[lo_mask],
        ref_id[shift(lo_mask, [1, 0], cval=False, order=0)],
        ref_id[shift(lo_mask, [1, 1], cval=False, order=0)],
    ], 1)
    faces_up_tri = np.stack([
        ref_id[up_mask],
        ref_id[shift(up_mask, [1, 1], cval=False, order=0)],
        ref_id[shift(up_mask, [0, 1], cval=False, order=0)],
    ], 1)
    faces_ma_tri = np.stack([
        ref_id[ma_mask],
        ref_id[shift(ma_mask, [1, 0], cval=False, order=0)],
        ref_id[shift(ma_mask, [0, 1], cval=False, order=0)],
    ], 1)
    faces = np.concatenate([faces_lo_tri, faces_up_tri, faces_ma_tri])

    return cor_id, door_x_resized, door_y_resized, us, vs, xs, ys, zs

def get_vertices(cor_id, xs, ys, zs):
    cor_id = cor_id.tolist()
    points = np.empty((0, 3))
    for point in cor_id:
        x = xs[point[0], point[1]]
        y = ys[point[0], point[1]]
        z = zs[point[0], point[1]]
        points = np.vstack((points, np.array([x, y, z])))
    return points

def Getdoor(x, y, xs, ys, zs):
    door_coorx = xs[x, y]
    door_coory = ys[x, y]
    door_coorz = zs[x, y]
    vertex_i = np.array([door_coorx, door_coory, door_coorz])

    door_coorx = xs[x, y-1]
    door_coory = ys[x, y-1]
    door_coorz = zs[x, y-1]
    vertex_j = np.array([door_coorx, door_coory, door_coorz])

    door_coorx = xs[x+1, y]
    door_coory = ys[x+1, y]
    door_coorz = zs[x+1, y]
    vertex_k = np.array([door_coorx, door_coory, door_coorz])

    door_coorx, door_coory, door_coorz = vertex_i

    v1 = np.array(vertex_j) - np.array(vertex_i)
    v2 = np.array(vertex_k) - np.array(vertex_i)
    normal = np.cross(v1, v2)
    normal = normal / np.linalg.norm(normal)
    return door_coorx, door_coory, door_coorz, normal

In [12]:
def transition(points1, coorx1, coory1, coorz1, points2, coorx2, coory2, coorz2):
  trans_matrix = np.array([coorx1, coory1, coorz1])-np.array([coorx2, coory2, coorz2])
  points2 = points2+trans_matrix
  coorx2 = coorx2+trans_matrix[0]
  coory2 = coory2+trans_matrix[1]
  coorz2 = coorz2+trans_matrix[2]
  return points2, coorx2, coory2, coorz2

In [13]:
def rotate(points1, vector1, points2, vector2):
    # Define the vectors
    vector1 = -vector1
    # Calculate the cross product between the two vectors
    cross_product = np.cross(vector1, vector2)

    # Calculate the dot product between the two vectors
    dot_product = np.dot(vector1, vector2)

    # Calculate the norm of the cross product
    cross_norm = np.linalg.norm(cross_product)

    # Calculate the rotation angle
    angle = np.arctan2(cross_norm, dot_product)

    # Calculate the rotation axis
    axis = cross_product / cross_norm

    # Create the rotation matrix
    rotation_matrix = np.array([[np.cos(angle) + axis[0]**2*(1-np.cos(angle)), 
                                axis[0]*axis[1]*(1-np.cos(angle)) - axis[2]*np.sin(angle), 
                                axis[0]*axis[2]*(1-np.cos(angle)) + axis[1]*np.sin(angle)],
                                [axis[1]*axis[0]*(1-np.cos(angle)) + axis[2]*np.sin(angle), 
                                np.cos(angle) + axis[1]**2*(1-np.cos(angle)), 
                                axis[1]*axis[2]*(1-np.cos(angle)) - axis[0]*np.sin(angle)],
                                [axis[2]*axis[0]*(1-np.cos(angle)) - axis[1]*np.sin(angle), 
                                axis[2]*axis[1]*(1-np.cos(angle)) + axis[0]*np.sin(angle), 
                                np.cos(angle) + axis[2]**2*(1-np.cos(angle))]])

    # Apply the rotation matrix to vector2
    new_vector2 = np.dot(rotation_matrix, vector2)

    print("Original vector1:", vector1)
    print("Original vector2:", vector2)
    print("Rotated vector2:", new_vector2)

    # Rotate points2 to be the same orientation as points1
    new_points2 = np.dot(points2, rotation_matrix)

    return new_points2, new_vector2


In [14]:
def generate_mesh_faces(n):
    i = np.arange(n)
    j = np.roll(i, -1)
    k = np.roll(i, -2)
    faces = np.column_stack((i, j, k))
    faces = np.column_stack(faces, )
    return faces


In [15]:
cor_id1, room1_x_resized, room1_y_resized, us1, vs1, xs1, ys1, zs1 = Infer(rgb1, 290, 526, net)
points1 = get_vertices(cor_id1, xs1, ys1, zs1)
coorx1, coory1, coorz1, normal1 = Getdoor(room1_x_resized, room1_y_resized, xs1, ys1, zs1)
faces1 = generate_mesh_faces(len(points1))
print(coorx1, coory1, coorz1)
print(normal1)


Eps time: 6.71 sec.
290 526
-0.28604719586724914 -3.0991155541336277 -0.6788553843753702
[-0.00301098  0.99999079 -0.00305786]


In [16]:
cor_id2, room2_x_resized, room2_y_resized, us2, vs2, xs2, ys2, zs2 = Infer(rgb2, 295, 960, net) # 문 좌표 업데이트 원래 중심: 295, 934
points2 = get_vertices(cor_id2, xs2, ys2, zs2)
coorx2, coory2, coorz2, normal2 = Getdoor(room2_x_resized, room2_y_resized, xs2, ys2, zs2)
faces2 = generate_mesh_faces(len(points2))
print(coorx2, coory2, coorz2)
print(normal2)
points2, norm2 = rotate(points1, normal1, points2[:,:3], normal2)
print("before transition : ", coorx2, coory2, coorz2)
points2, coorx2, coory2, coorz2 = transition(points1, coorx1, coory1, coorz1, points2[:,:3], coorx2, coory2, coorz2)
print("after transition : ", coorx2, coory2, coorz2)

Eps time: 3.24 sec.
295 960
-1.1106622119575442 2.728611397002909 -0.7187521657566527
[ 0.00332344 -0.9999904  -0.0028539 ]
Original vector1: [ 0.00301098 -0.99999079  0.00305786]
Original vector2: [ 0.00332344 -0.9999904  -0.0028539 ]
Rotated vector2: [ 0.00363578 -0.99995497 -0.00876555]
before transition :  -1.1106622119575442 2.728611397002909 -0.7187521657566527
after transition :  -0.28604719586724914 -3.0991155541336277 -0.6788553843753702


In [20]:
cor_id3, room3_x_resized, room3_y_resized, us3, vs3, xs3, ys3, zs3 = Infer(rgb3, 280, 900, net)                                                      # 3버어어어ㅓ어어언
points3 = get_vertices(cor_id3, xs3, ys3, zs3)
coorx3, coory3, coorz3, normal3 = Getdoor(room3_x_resized, room3_y_resized, xs3, ys3, zs3)
faces3 = generate_mesh_faces(len(points3))
print(coorx3, coory3, coorz3)
print(normal3)
# points3, norm3 = rotate(points3, normal1, points2[:,:3], normal2)
# print("before transition : ", coorx2, coory2, coorz2)
# points2, coorx2, coory2, coorz2 = transition(points1, coorx1, coory1, coorz1, points2[:,:3], coorx2, coory2, coorz2)
# print("after transition : ", coorx2, coory2, coorz2)

Eps time: 2.57 sec.
280 900
-1.2036021883207084 1.2798164307546982 -0.27163426946765895
[ 0.99999374  0.00284824 -0.00209879]


In [23]:
Mesh1 = go.Mesh3d(
            x=points1[:,0],
            y=points1[:,1],
            z=points1[:,2],
            i=faces1[:,0],
            j=faces1[:,1],
            k=faces1[:,2],
            )
Mesh2 = go.Mesh3d(
                 x=points2[:,0],
                 y=points2[:,1],
                 z=points2[:,2],
                 i=faces2[:,0],
                 j=faces2[:,1],
                 k=faces2[:,2],
                 )
fig = go.Figure( # Mesh1, Mesh2 빼놓음
    data=[
        # # Room1
        go.Scatter3d(
            x=[coorx1],
            y=[coory1],
            z=[coorz1],
            mode='markers',
            marker=dict(
                size=5,
                color='red'
            )
        ),
         go.Cone(x=[coorx1], y=[coory1], z=[coorz1],
                 u=[normal1[0]], v=[normal1[1]], w=[normal1[2]],
                 sizemode='absolute', sizeref=0.5,
                 anchor='tail', showlegend=False)
        # #  Room2
        #  
        #      go.Scatter3d(
        #          x=[coorx2],
        #          y=[coory2],
        #          z=[coorz2],
        #          mode='markers',
        #          marker=dict(
        #              size=5,
        #              color='red'
        #          )
        #      ),
        #      go.Cone(x=[coorx2], y=[coory2], z=[coorz2],
        #              u=[normal2[0]], v=[normal2[1]], w=[normal2[2]],
        #              sizemode='absolute', sizeref=0.5,
        #              anchor='tail', showlegend=False),
        #   # #  Room3
        #  go.Mesh3d(
        #          x=points3[:,0],
        #          y=points3[:,1],
        #          z=points3[:,2],
        #          i=faces3[:,0],
        #          j=faces3[:,1],
        #          k=faces3[:,2],
        #          facecolor=room3_points[:,3:][room3_faces[:,0]]
        #      ),
        #      go.Scatter3d(
        #          x=[coorx3],
        #          y=[coory3],
        #          z=[coorz3],
        #          mode='markers',
        #          marker=dict(
        #              size=5,
        #              color='red'
        #          )
        #      ),
        #      go.Cone(x=[coorx3], y=[coory3], z=[coorz3],
        #              u=[normal3[0]], v=[normal3[1]], w=[normal3[2]],
        #              sizemode='absolute', sizeref=0.5,
        #              anchor='tail', showlegend=False)
        ],
    layout=dict(
        scene=dict(
            xaxis=dict(title='X', visible=True),
            yaxis=dict(title='Y', visible=True),
            zaxis=dict(title='Z', visible=True),
            aspectmode="manual",
            aspectratio=dict(x=0.625, y=1, z=0.25),

        )
    )
)
fig.show()

In [24]:
def transform_open3d(plotly_mesh):
  points = np.column_stack((plotly_mesh.x, plotly_mesh.y, plotly_mesh.z))
  faces = np.column_stack((plotly_mesh.i, plotly_mesh.j, plotly_mesh.k))
  # Prepare vertices and faces
  vertices = o3d.utility.Vector3dVector(points[:, :3])
  # colors = o3d.utility.Vector3dVector(points[:, 3:] / 255.0)
  triangles = o3d.utility.Vector3iVector(points[:,:3])

  # Create mesh object
  mesh = o3d.geometry.TriangleMesh()
  meshvertices = vertices
  # mesh1.vertex_colors = colors
  mesh.triangles = triangles
  mesh.compute_vertex_normals()
  return mesh



In [ ]:
mesh1 = transform_open3d(Mesh1)
mesh2 = transform_open3d(Mesh2)



# Visualize mesh
o3d.visualization.draw_plotly([mesh2])